In [ ]:
pip install transformers

In [ ]:
import pandas as pd
import os
import numpy as np

In [ ]:
pip install tokenizers

In [ ]:
import os
from tokenizers.models import BPE
from tokenizers import Tokenizer
from tokenizers.decoders import ByteLevel as ByteLevelDecoder
from tokenizers.normalizers import NFKC, Sequence
from tokenizers.pre_tokenizers import ByteLevel
from tokenizers.trainers import BpeTrainer as BpeTrainer

In [ ]:
class BPE_token(object):
  def __init__(self):
    self.tokenizer=Tokenizer(BPE())
    self.tokenizer.normalizer=Sequence([NFKC()])
    self.tokenizer.pre_tokenizer=ByteLevel()
    self.tokenizer.decoder=ByteLevelDecoder()
  def bpe_train(self,paths):
    trainer=BpeTrainer(vocab_size= 50257,show_progress=True,initial_alphabet=ByteLevel.alphabet(),special_tokens=['[BEYT_B]','[BEYT_E]','[MESRA_SEP]','[POEM_B]','[PAD]','[UNK]','[MSK]','[NO_SEP]','[POEM_E]'])
    self.tokenizer.train(trainer,paths)
  def save_tokenizer(self,location,prefix=None):
    if not os.path.exists(location):
      os.makedirs(location)
    self.tokenizer.model.save(location,prefix)

به دلیل جلوگیری از شلوغ شدن کد قبلاً کدی که با آن تگ ها را به دیتاست اضافه می کردم حذف کردم و به همین دلیل کد آخرین روش اضافه کردن تگ ها در دیتاست موجود نیست! 




In [ ]:
from pathlib import Path
import os# the folder 'text' contains all the files
paths = [str(x) for x in Path("drive/MyDrive/poemDatasets_Second/train/").glob("**/*.txt")]
tokenizer = BPE_token()# train the tokenizer model
tokenizer.bpe_train(paths)# saving the tokenized data in our specified folder 
save_path = 'drive/MyDrive/tokenized_data_Second/train/'
tokenizer.save_tokenizer(save_path)

In [ ]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
from transformers import GPT2Config, TFGPT2LMHeadModel, GPT2Tokenizer# loading tokenizer from the saved model path
save_path = 'drive/MyDrive/tokenized_data_Fourth/'
tokenizer = GPT2Tokenizer.from_pretrained(save_path)
tokenizer.add_special_tokens({
    'bos_token':'[POEM_B]',
    'eos_token':'[POEM_E]',
   "unk_token":'[UNK]',
   "pad_token":'[PAD]',
   "mask_token":'[MSK]'
})# creating the configurations from which the mo

config = GPT2Config(
  vocab_size=tokenizer.vocab_size,
  bos_token_id=tokenizer.bos_token_id,
  eos_token_id=tokenizer.eos_token_id,
  pad_token_id=tokenizer.pad_token_id,
  unk_token_id=tokenizer.unk_token_id,
  mask_token_id=tokenizer.mask_token_id,
  n_positions=512,
  n_ctx= 512,
  n_embd=768

)# creating the model
model = TFGPT2LMHeadModel(config)
print(model.config)

GPT2Config {
  "activation_function": "gelu_new",
  "attn_pdrop": 0.1,
  "bos_token_id": 3,
  "embd_pdrop": 0.1,
  "eos_token_id": 8,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "mask_token_id": 6,
  "model_type": "gpt2",
  "n_ctx": 512,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 512,
  "pad_token_id": 4,
  "resid_pdrop": 0.1,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "transformers_version": "4.3.3",
  "unk_token_id": 5,
  "use_cache": true,
  "vocab_size": 50000
}



In [ ]:
class my_dictionary(dict): 
  
    # __init__ function 
    def __init__(self): 
        self = dict()      
    # Function to add key:value 
    def add(self, key, value):
        if key in self.keys(): 
          self[key].append( value)
        else:
          self[key]=[]
          self[key].append( value)
  
# Main Function 
dict_poems_train = my_dictionary() 
all_poems=[]
all_poems_hAndF=[];
with open('drive/MyDrive/poemDatasets_Fourth/test/AllPoemsTest.txt', "r", encoding='utf-8') as f:
  all_poems=f.readlines()
for i in range(0,len(all_poems)-1 ,2):
  all_poems[i]=all_poems[i].replace('\n','')
  all_poems[i+1]=all_poems[i+1].replace('\n','')
  dict_poems_train.add(all_poems[i],all_poems[i+1])
  if all_poems[i]=='حافظ' or all_poems[i]=='فردوسی':
    all_poems_hAndF.append(all_poems[i+1])

In [ ]:
dict_poems_train.keys()

dict_keys(['حافظ', 'فردوسی', 'مولوی', 'نظامی', 'سعدی', 'پروین اعتصامی', 'عطار', 'سنایی', 'وحشی', 'فرخی سیستانی', 'اوحدی', 'انوری', 'عراقی', 'صائب تبریزی', 'شیخ محمود شبستری', 'هاتف اصفهانی', 'منوچهری', 'باباطاهر', 'محتشم کاشانی', 'شیخ بهایی', 'سیف فرغانی', 'فروغی بسطامی', 'امیرخسرو دهلوی', 'رودکی', 'فخرالدین اسعد گرگانی', 'سلمان ساوجی', 'رهی معیری', 'اقبال لاهوری', 'کسایی', 'عرفی شیرازی', 'رضی\u200cالدین آرتیمانی', 'هلالی جغتایی', 'شاه نعمت\u200cالله ولی', 'شهریار', 'اسدی توسی', 'خیام', 'شاطرعباس صبوحی', 'ملک\u200cالشعرای بهار', 'مسعود سعد سلمان', 'جامی', 'رشیدالدین میبدی', 'عنصری', 'سعدالدین وراوینی', 'کمال\u200cالدین اسماعیل', 'حکیم نزاری', 'قدسی مشهدی', 'کمال خجندی', 'صامت بروجردی', 'نیما یوشیج', 'احمد شاملو', 'سهراب سپهری', 'فروغ فرخزاد', 'سیمین بهبهانی', 'مهدی اخوان ثالث', 'محمدحسن بارق شفیعی', 'عبدالواسع جبلی', 'عبدالقهّار عاصی', 'فریدون مشیری', 'سایه', 'نادر نادرپور', 'امام خمینی', 'احمد پروین', 'نجمه زارع', 'سید حمیدرضا برقعی', 'مولانا خالد نقشبندی', 'داوود ملک\u200cزاده', 'شیو

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
sumLen=0
numPoems=0
poems=''
for poem in all_poems_hAndF:
  poems+=poem+' '
string_tokenized = tokenizer.encode(poems)

In [ ]:
from pathlib import Path
single_string = ''
paths1 = [str(x) for x in Path("drive/MyDrive/poemDatasets_Fourth/train/").glob("**/*.txt")]
k=0
for filename in paths1: 
   with open(filename, "r", encoding='utf-8') as f:
    x = f.read()  
    single_string += x 
string_tokenized = tokenizer.encode(single_string)


In [ ]:
examples = []
block_size = 200
BATCH_SIZE = 12
BUFFER_SIZE = 1000
for i in range(0, len(string_tokenized) - block_size + 1, block_size):
  examples.append(string_tokenized[i:i + block_size])
inputs, labels = [], []
for ex in examples:
  inputs.append(ex[:-1])
  labels.append(ex[1:])
print(np.shape(inputs))
# dataset = tf.data.Dataset.from_tensor_slices((inputs,labels))
dataset = tf.data.Dataset.from_tensor_slices((inputs,labels))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

(62288, 199)


In [ ]:
# defining our optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)# definining our loss function
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)# defining our metric which we want to observe
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')# compiling the model
model.compile(optimizer=optimizer, loss=[loss, *[None] * model.config.n_layer], metrics=[metric])
# model.build(np.shape(dataset))
print(model.config)


GPT2Config {
  "activation_function": "gelu_new",
  "attn_pdrop": 0.1,
  "bos_token_id": 3,
  "embd_pdrop": 0.1,
  "eos_token_id": 8,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "mask_token_id": 6,
  "model_type": "gpt2",
  "n_ctx": 512,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 512,
  "pad_token_id": 4,
  "resid_pdrop": 0.1,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "transformers_version": "4.3.3",
  "unk_token_id": 5,
  "use_cache": true,
  "vocab_size": 50000
}



In [ ]:
num_epoch = 10
checkpoint_path = "drive/MyDrive/LU-HW_HF/cp-{epoch:04d}.ckpt"

checkpoint_dir = os.path.dirname(checkpoint_path)
print(checkpoint_dir)
latest = tf.train.latest_checkpoint(checkpoint_dir)
print(latest)
model.load_weights(latest)
batch_size = 12
# Create a callback that saves the model's weights every 5 epochs
# cp_callback = tf.keras.callbacks.ModelCheckpoint(
#     filepath=checkpoint_path, 
#     verbose=1, 
#     save_weights_only=True,
#     save_freq=5190)
# model.save_weights(checkpoint_path.format(epoch=0))
# history = model.fit(dataset,epochs=num_epoch,callbacks=[cp_callback])
# model.summary()

drive/MyDrive/LU-HW_HF
drive/MyDrive/LU-HW_HF/cp-0010.ckpt


In [ ]:
num_epoch = 6
checkpoint_path = "drive/MyDrive/training_10/cp-{epoch:04d}.ckpt"

checkpoint_dir = os.path.dirname(checkpoint_path)
print(checkpoint_dir)
latest = tf.train.latest_checkpoint(checkpoint_dir)
print(latest)
model.load_weights(latest)
batch_size = 12

# Create a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    save_freq=8305)
model.save_weights(checkpoint_path.format(epoch=11))
# outputs= model.
# history = model.fit(dataset,batch_size=12,epochs=num_epoch,callbacks=[cp_callback])
history = model.fit(dataset,epochs=num_epoch,callbacks=[cp_callback])
# model.summary()

In [ ]:
class my_dictionary(dict): 
  
    # __init__ function 
    def __init__(self): 
        self = dict()      
    # Function to add key:value 
    def add(self, key, value):
        if key in self.keys(): 
          self[key].append( value)
        else:
          self[key]=[]
          self[key].append( value)
  
# Main Function 
dict_poems_train = my_dictionary() 
all_poems=[]
all_poems_hAndF=[];
with open('drive/MyDrive/poemDatasets_Fourth/test/AllPoemsTest.txt', "r", encoding='utf-8') as f:
  all_poems=f.readlines()
for i in range(0,len(all_poems)-1 ,2):
  all_poems[i]=all_poems[i].replace('\n','')
  all_poems[i+1]=all_poems[i+1].replace('\n','')
  dict_poems_train.add(all_poems[i],all_poems[i+1])
  if all_poems[i]=='حافظ' or all_poems[i]=='فردوسی':
    all_poems_hAndF.append(all_poems[i+1])

import csv
with open('drive/MyDrive/poems_dataset_test.csv', mode='w' , encoding='utf-8') as poem_file:
    poem_writer = csv.writer(poem_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL )
    count=0  
    poem_writer.writerow(['col1','col2','m-b','poet'])  
    for poet in dict_poems_train.keys():   
      poems=dict_poems_train[poet]
      for poem in poems:
        beyts=poem.replace('[POEM_B]','').replace('[POEM_E]','').replace('[BEYT_B]','').split('[BEYT_E]')
        # print(beyts)
        for i in range(0,len(beyts)-1,1):
          if len(beyts[i+1])>1 and np.random.rand()<0.035:
             poem_writer.writerow([beyts[i],beyts[i+1],'b',poet])  
        for beyt in beyts:
          if len(beyt)>1:
            sep_word='[MESRA_SEP]'
            if '[NO_SEP]'in beyt:
              sep_word='[NO_SEP]'
            mesras=beyt.split(sep_word)
            for j in range(0,len(mesras)-1,1):
              if np.random.rand()<0.035:
                poem_writer.writerow([mesras[j],mesras[j+1],'m',poet])
        count+=1
        # if count>1:
        #   break
        



In [ ]:
import pandas as pd
data=pd.read_csv('drive/MyDrive/poems_dataset_test.csv')

poets=data['poet'].unique()

data[data['m-b']=='m'][data['poet']=='حافظ']


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


,col1,col2,m-b,poet
0,با لاله رخی و گوشه بستانی,عیشی بود آن نه حد هر سلطانی,m,حافظ
2,جز نقش تو در نظر نیامد ما را,جز کوی تو رهگذر نیامد ما را,m,حافظ
3,مشنو سخن خصم که بنشین و مرو,بشنو ز من این نکته که برخیز و بیا,m,حافظ
4,در باغ چو شد باد صبا دایهٔ گل,بربست مشاطه‌وار پیرایهٔ گل,m,حافظ
5,آصف عهد زمان جان جهان تورانشاه,که در این مزرعه جز دانهٔ خیرات نکشت,m,حافظ
...,...,...,...,...
6260,شیدای دو آهوی شکار اندازت,صد چون من و صد هزار چون استادم,m,حافظ
6262,خانم! منم که تلخ ترین واژه ی شما م,من،این کنیز گمشده در «چار راه» بخت,m,حافظ
6263,دستم نگیر زخمی ام از سر نوشت خود,آری نشسته بر دل من آه...! آه بخت,m,حافظ
6264,زنغمهٔ غم و اندوه ِ هق هق‌ات ای اشک,به لحظه لحظهٔ شعرم ترانه می‌باری,m,حافظ


In [ ]:

import numpy as np
from nltk.translate.bleu_score import sentence_bleu
dict_bleu_mesra={}
with open('drive/MyDrive/poems_bleu.txt', mode='w' , encoding='utf-8') as poem_bleu:
  for poet in poets:
    if poet=='حافظ' or poet=='فردوسی':
      data_ref=data[data['m-b']=='b'][data['poet']==poet]
      print(poet)
      poem_bleu.write(poet+'\n')
      sum_bleu=0
      num_dat=0
      for i in range(len(data_ref)):
        row=data_ref.iloc[i]
        input=row['col1']
        input=input[0:len(input)-1]
        input= input.replace('[MESRA_SEP]','')
        reference=row['col2'].replace('[MESRA_SEP]','').replace('  ',' ').replace('  ',' ').split(' ')
        input_ids = tokenizer.encode(input, return_tensors='tf')# getting out output
        # print(input_ids)
        # print(input_ids.shape[1])
        beam_output = model.generate(
          input_ids,
          max_length = input_ids.shape[1]*2 ,
          num_beams = 5,
          temperature = 0.3,
          no_repeat_ngram_size=3,
          num_return_sequences=5
        )
        # print(input_ids.shape[1]*3)
        candidate=tokenizer.decode(beam_output[0]).replace(input,'').replace('[BEYT_E]',' ').replace('[BEYT_B]',' ').replace('[MESRA_SEP]',' ').replace('  ',' ').replace('\t','').replace('[BEYT_B]','').replace('[BEYT_E]','').split(' ')
        ref1=[]
        for ref in reference:
          if len(ref)>0:
            ref1.append(ref)
        cand1=[]
        for cand in candidate:
          if len(cand)>0:
            cand1.append(cand)
        score=sentence_bleu([ref1],cand1)
        # print(input)
        # print(ref1)
        # print(cand1)
        # print(score)
        sum_bleu+=score
        num_dat+=1
        if num_dat%10==0:
          print(num_dat)
          print(sum_bleu)
          poem_bleu.write(str(score))
          poem_bleu.write('\n')
      poem_bleu.write(str(score))
      poem_bleu.write('\n')
      dict_bleu_mesra[poet]= sum_bleu/num_dat

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


حافظ


/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


10
0.5992632158286924
20
3.9021436914959096
30
6.817306216646528
40
7.7540177416162
50
10.041475675666925
60
11.17396234662747
70
12.60480015694009
80
13.682627564165278


/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


90
15.271161881148414
100
16.287986698769974
110
18.41442210301647
120
20.232408916179402
130
21.615942253521354
140
23.47276923017248
150
24.91666790005504
160
27.74027280721701
170
30.245280570939066
180
32.6309900805944
190
34.64189551561537
200
36.395037188328295
210
39.16388266510243
220
40.91870874308225
230
43.17949528561903
240
45.50713620536615
250
46.742617492000804
260
49.33128214902821
270
52.19130533910188
280
54.04221563837093
290
55.82174978871104
300
57.03312345486662
310
59.446553469036104
320
61.934523516277125
330
63.62148075767613
340
66.19609437093025
350
68.62326710833852
360
70.92540133388532
370
72.05935179970022
380
73.04544335005819
390
75.97808538288888
400
77.73078671995049
410
79.89196530745394
420
82.2974132640226
430
84.38317727604336
440
85.62926505272897
450
87.49139478883896
460
89.53607168521381
470
92.74015045319088
480
95.54096485141156
490
97.60318130173378
500
100.26567611329858
510
102.33953777304023
520
104.6873318480809
530
107.46420061395311
5

In [ ]:
import numpy as np
from nltk.translate.bleu_score import sentence_bleu
for poet in poets:
  if poet=='فردوسی':
    data_ref=data[data['m-b']=='b'][data['poet']==poet]
    print(poet)
    print(len(data_ref))
    for i in range(len(data_ref)):
      if np.random.rand() < 0.02:
        row=data_ref.iloc[i]
        input=row['col1']
        input=input[0:len(input)-1]
        input= input.replace('[MESRA_SEP]','')
        reference=row['col2']
        input_ids = tokenizer.encode(input, return_tensors='tf')# getting out output
        # print(input_ids)
        # print(input_ids.shape[1])
        beam_output = model.generate(
          input_ids,
          max_length = input_ids.shape[1]*2 ,
          num_beams = 5,
          temperature = 0.3,
          no_repeat_ngram_size=3,
          num_return_sequences=5
        )
      # print(input_ids.shape[1]*3)
        print(input)
        print(reference)
        print('outputs: ')
        for out in beam_output:
          candidate=tokenizer.decode(out)
          print(candidate)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """


فردوسی
266
  چو مهراب شد شاد و روشن روان   لبش گشت خندان و دل شادمان
  گرانمایه سیندخت را پیش خواند [MESRA_SEP]  بسی خوب گفتار با او براند 
outputs: 
  چو مهراب شد شاد و روشن روان   لبش گشت خندان و دل شادمان شد [MESRA_SEP] چو از مادر به خواب آمد ، ز درد
  چو مهراب شد شاد و روشن روان   لبش گشت خندان و دل شادمان شد [MESRA_SEP] چو ماه فلک، خورشید رخشان، خرم و
  چو مهراب شد شاد و روشن روان   لبش گشت خندان و دل شادمان شد [MESRA_SEP] چو چشم مست او، امید وصال یار من
  چو مهراب شد شاد و روشن روان   لبش گشت خندان و دل شادمان شد [MESRA_SEP] چو ماه فلک، خورشید رخشان، خرم از
  چو مهراب شد شاد و روشن روان   لبش گشت خندان و دل شادمان شد [MESRA_SEP] چو از مادر به خواب آمد ، به آهنگ
  چو بشنید گفتار گودرز شاه   بدانست کاو دارد آیین و راه
  پشیمان بشد زان کجا گفته بود [MESRA_SEP]  بیهودگی مغزش آشفته بود 
outputs: 
  چو بشنید گفتار گودرز شاه   بدانست کاو دارد آیین و راه روزن [MESRA_SEP] به تیر و تیغ و تیر و
  چو بشنید گفتار گودرز شاه   بدانست کاو دارد آیین و راه [MESRA_SEP] به کردار دل اهل دلان شد ، به


In [ ]:
dict_bleu_mesra

{'حافظ': 0.20598327859442975, 'فردوسی': 0.15421370843466456}

In [ ]:
from nltk.translate.bleu_score import sentence_bleu

In [ ]:
text='گفتم غم تو دارم گفتا غمت سر آید'
input_ids = tokenizer.encode(text, return_tensors='tf')# getting out output
print(input_ids)
beam_output = model.generate(
  input_ids,
  max_length = input_ids.shape[1]*5,
  num_beams = 5,
  temperature = 0.3,
  no_repeat_ngram_size=3,
  num_return_sequences=5
)
for output in  beam_output:
  print(tokenizer.decode(output))

In [ ]:
class my_dictionary(dict): 
  
    # __init__ function 
    def __init__(self): 
        self = dict()      
    # Function to add key:value 
    def add(self, key, value):
        if key in self.keys(): 
          self[key].append( value)
        else:
          self[key]=[]
          self[key].append( value)
  
# Main Function 
dict_poems_train = my_dictionary() 

In [ ]:

all_poems=[]
with open('drive/MyDrive/poemDatasets_Fourth/train/AllPoemsTrain.txt', "r", encoding='utf-8') as f:
  all_poems=f.readlines()
for i in range(0,len(all_poems)-1 ,2):
  all_poems[i]=all_poems[i].replace('\n','')
  all_poems[i+1]=all_poems[i+1].replace('\n','')
  dict_poems_train.add(all_poems[i],all_poems[i+1])


In [ ]:
print(len(dict_poems_train.keys()))

92


In [ ]:
count=0
for key in dict_poems_train.keys():
  print(key)

In [ ]:
!nvidia-smi

Wed Feb 24 20:19:10 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    34W / 250W |   8579MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------